# About 

The idea is discussed [here, method 2](https://www.kaggle.com/c/siim-covid19-detection/discussion/245323), that is to add segmented roi or the predicted segmentation maps from segmentation model as an additional channel of model inputs. However, there are three possible ways to achieve this in terms of **Spatial** and **Channel** supervised approach, such as 

1. **Channel Supervision**: Use Segmentaiton model - get predictin maps - add it with inputs (multi-channel, 2 or 4). The classifier input can be 2 channel inpuut or 4 channel, depends. If choose either case, you may not use pre-trained weights directly but need to add one 3 filter conv layer with consistant padding. Also, we can drop 1 channel out of RGB and add the segmentatoin maps to complete 3 channel classifier input for pre-trained weights.

![one](https://user-images.githubusercontent.com/17668390/122654678-e6940480-d16e-11eb-8db7-38fd7da7851b.png)

2. **Spatial Supervision (1)**: Use Boundig-Box cordinates to draw rectangle on inputs. In the data loading time, draw bbox on the sample; maybe fill up with soft color probably can do some good. 


![two](https://user-images.githubusercontent.com/17668390/122654774-7a65d080-d16f-11eb-8f9c-2ea09a1c09d8.png)

---

3. **Spatial Supervision (2)**: Use Cropped-Roi-Mask, blend it with the original image to produce **in-place reflected precise spatial location**. To do that, first we will create a gaussian kernel and then convolves it on the mask image. Then we add the convolved output to the original image to produce final input samples. 

```python
kernel <- cv2.getGaussianKernel(ksize=ksize, sigma=sigma)
kernel2d <- np.dot(kernel, kernel.T)
msk <- convolve2d(msk, kernel2d, mode='same')
blended <- img + msk
```
![new](https://user-images.githubusercontent.com/17668390/122655916-17c50280-d178-11eb-9e30-64039bd035eb.png)


---

Here, we will be doing on approach 3, the **spatial supervison** by blending the cropped masks with corresponding x-ray samples to highlighten the precise location. It will be on the training time of course. 

In [ ]:
import os, math
import psutil, random 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import cv2; print(cv2.__version__)
import tensorflow as tf; print(tf.__version__) 

In [ ]:
MIXED_PRECISION = True
XLA_ACCELERATE  = False

GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for GPU in GPUS:
            tf.config.experimental.set_memory_growth(GPU, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(GPUS), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    except RuntimeError as  RE:
        print(RE)

if MIXED_PRECISION:
    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    tf.keras.mixed_precision.experimental.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

# Data Preprocess

In [ ]:
study_df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv'); print(study_df.shape)
study_df['StudyInstanceUID'] = study_df['id'].apply(lambda x: x.replace('_study', ''))
del study_df['id']

def hot_to_sparse(row):
    return(row.index[row.apply(lambda x: x==1)][0])
study_df['diagnosis'] = study_df.apply(lambda row:hot_to_sparse(row), axis=1)
cls = {
    'Typical Appearance':1,                    
    'Negative for Pneumonia':2,                
    'Indeterminate Appearance':3,                     
    'Atypical Appearance':4,    
}
study_df['sparse_gt'] = study_df.diagnosis.map(cls) 

image_df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv'); print(image_df.shape)
train = image_df.merge(study_df, on='StudyInstanceUID')
train['id'] = train['id'].apply(lambda x: x.replace('_image', ''))
display(train.head()); print(train.shape)

# ROI Segment: Cropped Bounding Box Mask Blending

In [ ]:
import scipy.stats as st
from scipy.signal import convolve2d

def blend(img, msk):
    '''
    https://gist.github.com/innat/56786d001048ccd4a68ad41e71869abb
    '''
    img = np.asarray(img, np.float32) / 255.
    msk = np.asarray(msk, np.float32) / 255.

    sigma = np.random.uniform(low=2, high=5)
    gamma = np.random.uniform(low=1.3, high=1.3)
    kernel = cv2.getGaussianKernel(ksize=3, sigma=sigma)
    kernel2d = np.dot(kernel, kernel.T)

    msk[...,0] = convolve2d(msk[...,0], kernel2d, mode='same')
    blended = img + msk

    if np.max(blended) > 1:
        m = blended[blended > 1]
        m = (np.mean(m) - 1) * gamma
        msk = np.clip(msk - m, 0, 1)
        blended = np.clip(msk + img, 0, 1)
    return blended

In [ ]:
def vis(path1, path2, n_images, is_random=True, figsize=(16, 16)):
    '''
    https://gist.github.com/innat/00de7561033ba373745d425c6da7bf8c
    '''
    image_names = os.listdir(path1)
    masks_names = os.listdir(path2)
    
    for i in range(n_images):
        if is_random:
            image_name = random.choice(masks_names)
            masks_name = image_name
        else:
            image_name = masks_names[i]
            masks_name = masks_names[i]
        
        img = cv2.resize(cv2.imread(os.path.join(path1, image_name)), (512, 512))
        msk = cv2.resize(cv2.imread(os.path.join(path2, masks_name)), (512, 512))
        bld = blend(img, msk)
        
        plt.figure(figsize=(15,15))
        plt.subplot(131); plt.imshow(img);
        plt.subplot(132); plt.imshow(msk);
        plt.subplot(133); plt.imshow(bld);
        plt.show()

In [ ]:
base_path = '../input/covid19-detection-890pxpng-study'
TRAIN_IMG_PATH =  os.path.join(base_path, 'train/')
TRAIN_MSK_PATH = os.path.join(base_path, 'ROI Mask/')
vis(TRAIN_IMG_PATH, TRAIN_MSK_PATH, 10, is_random=True)

In [ ]:
from sklearn.model_selection import GroupKFold

skf = GroupKFold(n_splits=3)
for index, (train_index, val_index) in enumerate(skf.split(train, groups = train.id.tolist())):
    train.loc[val_index, 'fold'] = index
    
print(train.groupby(['fold', train.sparse_gt]).size())

# Data Generator

In [ ]:
import albumentations as A 

# For Validation 
def albu_transforms_train(data_resize): 
    return A.Compose([
        A.Resize(data_resize, data_resize),
        A.ToFloat(), # no need if use keras.applicaiton.EfficientNets Bx
        A.RandomBrightnessContrast(brightness_limit=0.2,
                                   contrast_limit=0.2, p=0.75),
        A.CoarseDropout(max_holes=10, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1,
                           rotate_limit=15, border_mode=0, p=0.65)

    ], p=1.)

# For Validation 
def albu_transforms_valid(data_resize): 
    return A.Compose([
        A.Resize(data_resize, data_resize),
        A.ToFloat(), # no need if use keras.applicaiton.EfficientNets Bx
        ], p=1.)

In [ ]:
class Covid19Generator(tf.keras.utils.Sequence):
    def __init__(self, img_path, msk_path, data, batch_size, random_state, 
                 dim, shuffle=True, transform=None, is_train=False):
        self.dim = dim
        self.data = data
        self.random_state = random_state
        self.shuffle  = shuffle
        
        self.img_path = img_path
        self.msk_path = msk_path
        self.is_train = is_train
        
        self.augment  = transform
        self.batch_size = batch_size
        
        self.list_idx = data.index.values
        self.label = self.data[['Negative for Pneumonia', 
                                'Typical Appearance', 
                                'Indeterminate Appearance', 
                                'Atypical Appearance']] if self.is_train else np.nan
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_idx) / self.batch_size))
    
    def __getitem__(self, index):
        batch_idx = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        idx = [self.list_idx[k] for k in batch_idx]
        
        Data = np.zeros((self.batch_size,) + self.dim + (1,), dtype="float32")
        Mask = np.zeros((self.batch_size,) + self.dim + (1,), dtype="float32")
        Target = np.zeros((self.batch_size, 4), dtype = np.float32)

        for i, k in enumerate(idx):
            # load the image file using cv2
            image = cv2.imread(self.img_path + 
                               self.data['id'][k] + '.png', 0) 
            mask = cv2.imread(self.msk_path + 
                              self.data['id'][k] + '.png', 0)
            
            if mask is None:
                # no mask.png for no bounding box image 
                mask = np.zeros_like(image)
            
            if self.shuffle: # do blend only for training - not for validation
                image = self.blend(image[:, :, np.newaxis], 
                                   mask[:, :, np.newaxis]) # WARNING: blending can be expensive
            else:
                image = image[:, :, np.newaxis]
                
            res = self.augment(image=image)
            image = res['image']
            
            # assign 
            if self.is_train:
                Data[i,] = image
                Target[i,] = self.label.iloc[k,].values 
            else:
                Data[i,] =  image 
        
        return Data, Target
    
    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_idx))
        if self.shuffle:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indices)
            
    def blend(self, img, msk):
        img = np.asarray(img, np.float32) / 255.
        msk = np.asarray(msk, np.float32) / 255.

        sigma = np.random.uniform(low=2, high=5)
        gamma = np.random.uniform(low=1.3, high=1.3)
        kernel = cv2.getGaussianKernel(ksize=3, sigma=sigma)
        kernel2d = np.dot(kernel, kernel.T)

        msk[...,0] = convolve2d(msk[...,0], kernel2d, mode='same')
        blended = img + msk
        
        if np.max(blended) > 1:
            m = blended[blended > 1]
            m = (np.mean(m) - 1) * gamma
            msk = np.clip(msk - m, 0, 1)
            blended = np.clip(msk + img, 0, 1)
        return blended

In [ ]:
from pylab import rcParams

# helper function to plot sample 
def plot_imgs(dataset_show, row, col):
    rcParams['figure.figsize'] = 20,10
    for i in range(row):
        f, ax = plt.subplots(1,col)
        for p in range(col):
            idx = np.random.randint(0, len(dataset_show))
            img, label = dataset_show[idx]
            ax[p].grid(False)
            ax[p].imshow(img[0], cmap='gray')
            ax[p].set_title(label[0])
    plt.show()

In [ ]:
TRAIN_IMG_PATH = '../input/covid19-detection-890pxpng-study/train/'
TRAIN_MSK_PATH = '../input/covid19-detection-890pxpng-study/ROI Mask/'

batch_size = 86
size = 384
fold = 1

def count_data_items(length, b_max):
    batch_size = sorted([int(length/n) for n in range(1, length+1) \
                         if length % n == 0 and length/n <= b_max], reverse=True)[0]  
    steps  = length / batch_size 
    return batch_size, steps

def fold_generator(fold):
    # for way one - data generator
    train_labels = train[train.fold != fold].reset_index(drop=True)
    val_labels = train[train.fold == fold].reset_index(drop=True)

    train_generator = Covid19Generator(TRAIN_IMG_PATH, TRAIN_MSK_PATH,
                              train_labels, 
                              batch_size, 1234, (size, size),
                              shuffle = True, is_train = True,
                              transform = albu_transforms_train(size))
    
    valid_batch, valid_step = count_data_items(len(val_labels), batch_size)

    val_generator = Covid19Generator(TRAIN_IMG_PATH, TRAIN_MSK_PATH,
                              val_labels, 
                              valid_batch, 1234, (size, size),
                              shuffle = False, is_train = True,
                              transform = albu_transforms_valid(size))

    return train_generator, val_generator, train_labels, val_labels, valid_step

train_gen, val_gen, train_len, val_len, val_step = fold_generator(fold)

In [ ]:
plot_imgs(train_gen, 10, 4)

In [ ]:
plot_imgs(val_gen, 2, 4)

# Model 


In [ ]:
from tensorflow.keras import Model 
from tensorflow.keras import Sequential 
from tensorflow.keras import Input 
from tensorflow.keras import layers 
from tensorflow.keras import applications 

class CovidNet(Model):
    '''To handle different channel input for pre-trained weights 
    option 1: https://stackoverflow.com/a/67576025/9215780
    option 2: https://stackoverflow.com/a/67540516/9215780
    '''
    def __init__(self):
        super(CovidNet, self).__init__()
        self.base = applications.ResNet50(input_shape=(size, size, 1),
                                                      include_top=False,
                                                      weights=None)
        base_weights = applications.ResNet50(input_shape=(size, size, 3),
                                                         include_top=False,
                                                         weights='imagenet')

        for i in range(3, len(self.base.layers)):
            self.base.layers[i].set_weights(base_weights.layers[i].get_weights())

        del base_weights
        self.pool = layers.GlobalAveragePooling2D()
        self.out = layers.Dense(units=4, activation='softmax')

    def call(self, x, training=None, **kwargs):
        x = self.base(x)
        x = self.pool(x)
        x = self.out(x)
        return x


tf.keras.backend.clear_session()
model = CovidNet()
model.build(input_shape=(None, size, size, 1))
model.summary()

# Training

In [ ]:
from tensorflow.keras.optimizers.schedules import LearningRateSchedule, ExponentialDecay

class WarmupLearningRateSchedule(LearningRateSchedule):
    """Provides a variety of learning rate decay schedules with warm up."""

    def __init__(self,
               initial_lr,
               steps_per_epoch=None,
               lr_decay_type='exponential',
               decay_factor=0.97,
               decay_epochs=2.4,
               total_steps=None,
               warmup_epochs=5,
               minimal_lr=0):
        super(WarmupLearningRateSchedule, self).__init__()
        self.initial_lr = initial_lr
        self.steps_per_epoch = steps_per_epoch
        self.lr_decay_type = lr_decay_type
        self.decay_factor = decay_factor
        self.decay_epochs = decay_epochs
        self.total_steps = total_steps
        self.warmup_epochs = warmup_epochs
        self.minimal_lr = minimal_lr

    def __call__(self, step):
        if self.lr_decay_type == 'exponential':
            assert self.steps_per_epoch is not None
            decay_steps = self.steps_per_epoch * self.decay_epochs
            lr = ExponentialDecay(self.initial_lr, decay_steps, 
                                  self.decay_factor, staircase=True)(step)
        elif self.lr_decay_type == 'cosine':
            assert self.total_steps is not None
            lr = 0.5 * self.initial_lr * (
              1 + tf.cos(np.pi * tf.cast(step, tf.float32) / self.total_steps))
        elif self.lr_decay_type == 'linear':
            assert self.total_steps is not None
            lr = (1.0 - tf.cast(step, tf.float32) / self.total_steps) * self.initial_lr
        elif self.lr_decay_type == 'constant':
            lr = self.initial_lr
        else:
            assert False, 'Unknown lr_decay_type : %s' % self.lr_decay_type

        if self.minimal_lr:
            lr = tf.math.maximum(lr, self.minimal_lr)

        if self.warmup_epochs:
            warmup_steps = int(self.warmup_epochs * self.steps_per_epoch)
            warmup_lr = (
              self.initial_lr * tf.cast(step, tf.float32) /
              tf.cast(warmup_steps, tf.float32))
            lr = tf.cond(step < warmup_steps, lambda: warmup_lr, lambda: lr)

        return lr

    def get_config(self):
        return {
            'initial_lr': self.initial_lr,
            'steps_per_epoch': self.steps_per_epoch,
            'lr_decay_type': self.lr_decay_type,
            'decay_factor': self.decay_factor,
            'decay_epochs': self.decay_epochs,
            'total_steps': self.total_steps,
            'warmup_epochs': self.warmup_epochs,
            'minimal_lr': self.minimal_lr,
        }

In [ ]:
steps_per_epoch  = np.ceil(float(len(train_len)) / batch_size) 
validation_steps = val_step 
epochs = 10

lr_sched = 'cosine'
lr_base = 0.016
lr_min=0
lr_decay_epoch = 2.4
lr_warmup_epoch = 5
lr_decay_factor = 0.97

scaled_lr = lr_base * (batch_size / 256.0)
scaled_lr_min = lr_min * (batch_size / 256.0)
total_steps = steps_per_epoch * epochs

learning_rate = WarmupLearningRateSchedule(
    scaled_lr,
    steps_per_epoch=steps_per_epoch,
    decay_epochs=lr_decay_epoch,
    warmup_epochs=lr_warmup_epoch,
    decay_factor=lr_decay_factor,
    lr_decay_type=lr_sched,
    total_steps=total_steps,
    minimal_lr=scaled_lr_min)

In [ ]:
from tensorflow.keras import metrics
from tensorflow.keras import losses 
from tensorflow.keras import optimizers

# bind all
model.compile(
    loss = losses.CategoricalCrossentropy(),
    metrics = [metrics.SensitivityAtSpecificity(0.4, name='@specificity'), 
               metrics.SpecificityAtSensitivity(0.4, name='@sensitivity'),
               metrics.AUC(curve='PR', summation_method='interpolation')],
    optimizer = optimizers.Adam(learning_rate))


# list of call backs 
from tensorflow.keras import callbacks
callback_list = [
       callbacks.ModelCheckpoint(
            filepath='model.{epoch:02d}-{val_loss:.4f}.h5', 
            save_freq='epoch', verbose=1, monitor='val_loss', 
            save_weights_only=True, save_best_only=True
       )         
]


# fitter 
model.fit(train_gen, 
          steps_per_epoch=steps_per_epoch,
          validation_data=val_gen, 
          validation_steps=validation_steps,
          callbacks=callback_list, 
          workers=psutil.cpu_count(),
          epochs=epochs)

# Resources 
- [Add Attention Mechansim](https://www.kaggle.com/ipythonx/tf-keras-ranzcr-multi-attention-efficientnet): Try to add attention function to build hybrid model. 
- [Out-of-Fold Evaluation](https://www.kaggle.com/ipythonx/optimizing-metrics-out-of-fold-weights-ensemble): Compute the oof each fold and compare the match and non-match prediction in order to emphasize on the weak or minor cases. 